In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/My Drive/Improving-critical-exponents_pytorch/')
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

class objectview(object):
    def __init__(self, d):
        self.__dict__ = d

cuda


In [ ]:
args_cnn = {
    'save_dir': './saved_models/cnn_regression',
    'lattice_size': 128,
    'dataset_size': 5000,
    'epochs': 100,
    'batch_size': 256,
    'dropout_rate': 0.0,
    'learning_rate': 10e-4,
    'device': device,
    'save_checkpoints': True,
}
        
args_cnn = objectview(args_cnn)

from src.cnn.train import main as main_cnn
main_cnn(args_cnn)

In [3]:
from src.cnn.network import CNN

CNN_model_path = "./saved_models/cnn_regression/2021.10.22.12.24.41/model/final_model.pt"
cnn_checkpoint = torch.load(CNN_model_path, map_location=torch.device(device))
cnn_checkpoint['constructor_args']['device'] = device
cnn = CNN(**cnn_checkpoint['constructor_args'])   
cnn.load_state_dict(cnn_checkpoint['model_state_dict'])
cnn = cnn.eval()

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [8]:
from src.cnn.data import generate_data_torch

X, y = generate_data_torch(dataset_size=100, 
                           lattice_size=128,
                           split=False,
                           save_dir=None)

X = X.to(device)
y = y.to(device)

In [9]:
torch.mean(torch.abs(cnn(X) - y))

RuntimeError: ignored

In [ ]:
args_gen = {
    'save_dir': './saved_models/gan_cnn_regression',
    'epochs': 10,
    'batch_size': 5,
    'learning_rate': 10e-3,
    'device': device,
    'bins_number': 100,
    'noise_dim': 100,
    'ckpt_freq': 10,
    'CNN_model_path': './saved_models/cnn_regression/2021.10.22.12.24.41/model/final_model.pt',
    'set_generate_plots': False,
    'regularization_strength': 0.5,
}

args_gen = objectview(args_gen)

from src.generator_cnn.train import main as main_gen
main_gen(args_gen)

In [ ]:
args_hydra = {
    'lattice_size': 128,
    'dataset_size': 1000,
    'batch_size': 128,
    'epochs': 150,
    'bins_number': 100,
    'generator_learning_rate': 10e-3,
    'discriminator_learning_rate': 10e-3,
    'n_conv_cells': 5,
    'n_convt_cells': 5,
    'regularization_strength': 1.0,
    'hydra_ratio_bce': 0.0,
    'hydra_ratio_cnn': 1.0,
    'patience_generator': 2, 
    'noise_dim': 100,
    'wanted_p': 0.5928,
    'save_dir': "./saved_models/hydra",
    'CNN_model_path': "./saved_models/cnn_regression/2021.10.22.12.24.41/model/final_model.pt",
    'device': device,
    'set_generate_plots': False,
}

args_hydra = objectview(args_hydra)

from src.hydra.train import main as main_hydra
main_hydra(args_hydra)

NameError: name 'device' is not defined